In [1]:
pip install azure-ai-formrecognizer azure-search-documents openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.4/301.4 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.9/198.9 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 5.9 MB/s eta 0:00:00


In [3]:
pip install azure-ai-documentintelligence --pre

In [5]:
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeResult, AnalyzeDocumentRequest

In [17]:
from azure.ai.formrecognizer import DocumentAnalysisClient
from azure.core.credentials import AzureKeyCredential
import os

Azure AI services | Document intelligence -
FormRecognizer resource

In [18]:
# Set up the Form Recognizer client
endpoint = "https://.cognitiveservices.azure.com/"
api_key = ""
doc_client = DocumentAnalysisClient(endpoint=endpoint, credential=AzureKeyCredential(api_key))


In [9]:
import os
os.listdir()

['.config', 'contract3.pdf', 'sample_data']

In [10]:
clause_titles = ["Price Protection", "Late Payment Penalties", "SLAs/Service Credits", "Termination", "Confidentiality", "Indemnification", "Rebates/Volume Discounts", "Payment Terms", "Forex", "COLA/Inflation Adjustment", "Early Payment Discounts", "Early Payment Discounts", "Termination Provisions", "Committal/Non-Committal Contracts"]


In [14]:
def get_words(page, line):
    result = []
    for word in page.words:
        if _in_span(word, line.spans):
            result.append(word)
    return result

In [15]:
def _in_span(word, spans):
    for span in spans:
        if word.span.offset >= span.offset and (word.span.offset + word.span.length) <= (span.offset + span.length):
            return True
    return False

In [32]:
def extract_clauses(pdf_path):
    with open(pdf_path, "rb") as fd:
        poller = doc_client.begin_analyze_document("prebuilt-layout", fd)
        result = poller.result()

    clause_segments = []
    current_label = None
    current_text = []

    for page in result.pages:
        for line in page.lines:
            line_text = line.content.strip()
            if len(line_text) == 1:
                continue
            # Check if paragraph text matches any clause title
            matched_title = next((title for title in clause_titles if title.lower() in line_text.lower()), None)

            if matched_title:
                # Save previous clause segment if there's any
                if current_label and current_text:
                    clause_segments.append({
                        "Label": current_label,
                        "Text": " ".join(current_text)
                    })

                # Reset for new clause
                current_label = matched_title
                current_text = [line_text]
            else:
                # Continue accumulating text for the current clause
                current_text.append(line_text)

    # Add the final clause to segments if it exists
    if current_label and current_text:
        clause_segments.append({
            "Label": current_label,
            "Text": " ".join(current_text)
        })

    return clause_segments

# Call the function to process the contract and get clause segments
file_path = "contract3.pdf"
clause_segments = extract_clauses(file_path)

# Check the output format
for clause in clause_segments:
    print(f"Label: {clause['Label']}, Text: {clause['Text'][:120]}...")  # Display first 100 characters of each clause


Label: Rebates/Volume Discounts, Text: 1. Rebates/Volume Discounts The Supplier agrees to provide volume-based discounts to the Buyer, incentivizing higher pur...
Label: SLAs/Service Credits, Text: 2. SLAs/Service Credits The Supplier commits to maintaining a minimum service level, as outlined in the Service Level Ag...
Label: Confidentiality, Text: Confidentiality Both parties agree not to disclose confidential information without written consent. 2024-01- 01 2025- 0...
Label: Termination, Text: Termination Either party may terminate the agreement with 30 days' written notice. 2024-01- 01 2025- 01-01 N/A $10,000 i...
Label: Price Protection, Text: 3. Price Protection The prices set forth in this agreement shall remain fixed for the initial 12 months. Thereafter, pri...


In [29]:

with open("contract3.pdf", "rb") as f:
    poller = doc_client.begin_analyze_document("prebuilt-layout", document=f, pages="1-6")
    result = poller.result()

# Extracting line content from each page
'''
for page in result.pages:

    if page.lines:
            for line_idx, line in enumerate(page.lines):
                words = get_words(page, line)
                if len(words) > 1:
                    print(f"...Line # {line_idx} has text '{line.content}' ")
'''


# Extract tables if available
tables = []
if hasattr(result, "tables"):
    for table in result.tables:
        table_data = []
        for cell in table.cells:
            table_data.append({
                "row": cell.row_index,
                "column": cell.column_index,
                "text": cell.content
            })
        tables.append(table_data)

# Output extracted tables
print("\nExtracted Tables:", tables)




Extracted Tables: [[{'row': 0, 'column': 0, 'text': 'Clause'}, {'row': 0, 'column': 1, 'text': 'Description'}, {'row': 0, 'column': 2, 'text': 'Effective Date'}, {'row': 0, 'column': 3, 'text': 'Expiry Date'}, {'row': 0, 'column': 4, 'text': 'Payment Terms'}, {'row': 0, 'column': 5, 'text': 'Penalties'}, {'row': 1, 'column': 0, 'text': 'Confidentiality'}, {'row': 1, 'column': 1, 'text': 'Both parties agree not to disclose confidential information without written consent.'}, {'row': 1, 'column': 2, 'text': '2024-01- 01'}, {'row': 1, 'column': 3, 'text': '2025- 01-01'}, {'row': 1, 'column': 4, 'text': 'N/A'}, {'row': 1, 'column': 5, 'text': 'Up to $50,000'}, {'row': 2, 'column': 0, 'text': 'Termination'}, {'row': 2, 'column': 1, 'text': "Either party may terminate the agreement with 30 days' written notice."}, {'row': 2, 'column': 2, 'text': '2024-01- 01'}, {'row': 2, 'column': 3, 'text': '2025- 01-01'}, {'row': 2, 'column': 4, 'text': 'N/A'}, {'row': 2, 'column': 5, 'text': '$10,000 if

tables = []
for table in result.tables:
    table_data = []
    for cell in table.cells:
        table_data.append({
            "row": cell.row_index,
            "column": cell.column_index,
            "text": cell.content
        })
    tables.append(table_data)

print(tables)

In [33]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential


In [34]:
import re
import openai
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchableField



Cognitive Search -> Azure AI services | AI Search - Search Service resouce.
Also manually creae an index with required fields

In [36]:
search_endpoint = "https://.search.windows.net"
search_api_key = ""
index_name = "contindex"
search_client = SearchClient(endpoint=search_endpoint, index_name=index_name, credential=AzureKeyCredential(search_api_key))


In [39]:
import json

In [41]:
def create_or_update_index(clause_segments):
    # Create a simple index for storing clause titles and content
    documents = []
    for i, clause in enumerate(clause_segments):
        document = {
                "id": f"clause-{i}",  # Unique ID for each clause
                "label": clause["Label"],
                "text": clause["Text"],
                "table": ""
        }
        documents.append(document)


    for table in tables:
        for cell in table:
            table_data = {
                "id": f"table_{cell['row']}_{cell['column']}",  # Unique ID for the table cell
                "label": "Table",  # Set the label appropriately
                "text": "",  # No text content for table cells
                "table": json.dumps(table)  # Store table data as JSON
            }
            documents.append(table_data)

    # Upload the document
    search_client.upload_documents(documents=documents)

# Index the extracted contract text
create_or_update_index(clause_segments)

In [42]:
# Query all documents in the index
def get_all_documents():
    results = search_client.search(search_text="*", top=10)  # Use "*" to match all documents
    for result in results:
        print(result)

# Call the function to print documents
get_all_documents()

{'id': 'clause-1', 'table': '', 'label': 'SLAs/Service Credits', 'text': "2. SLAs/Service Credits The Supplier commits to maintaining a minimum service level, as outlined in the Service Level Agreement (SLA) Schedule of this contract. This includes an uptime percentage of no less than 99.5% monthly. In the event of any service interruptions or downtimes exceeding the allowable limit, the Supplier will provide service credits to the Buyer based on the duration and impact of the downtime. Service credits will be calculated as a percentage of the monthly service fee, with specific percentages detailed in the SLA Schedule. Credits will be applied to the Buyer's next billing cycle. If the Supplier fails to meet the SLA for three consecutive months, the Buyer reserves the right to review the agreement and, if necessary, terminate the contract without penalty. Both parties agree that the Supplier will proactively report service performance at the end of each month, providing transparency and 

In [43]:
def search_for_clause(clause_title):
    results = search_client.search(clause_title)  # Search for a specific clause title
    clauses = []
    for result in results:
        clauses.append(result['text'])
    return clauses

# Search for a specific clause
clause_title = "Late Payment Penalties"
clauses = search_for_clause(clause_title)
print(clauses)

['8. Late Payment Penalties', "6. Payment Terms The Buyer agrees to settle all invoices within 30 days of receipt. Payments will be made via bank transfer to the Supplier's designated account. In exchange for early payment, the Supplier offers a 2% discount if the invoice is paid within 10 days. This clause incentivizes the Buyer to make early payments while offering a standard 30-day payment period. The early payment discount supports the Supplier's cash flow while providing savings for the Buyer.", '7. Early Payment Discounts The Buyer is entitled to a 2% discount on the total invoice amount if full payment is made within 10 days from the invoice issuance date. This discount will apply only to invoices paid via direct bank transfer and does not apply to partial payments. This clause encourages the Buyer to settle invoices promptly, improving cash flow for the Supplier. The direct bank transfer requirement ensures lower processing fees and quicker fund availability.', '', "2. SLAs/Ser

In [15]:
!pip install openai

In [44]:
import openai
import os

Create Azure OpenAI resource - GPT-4 deployment

Test env - GPT-35 deployment with its own resource

In [45]:
from openai import AzureOpenAI
client = AzureOpenAI(
  api_key="",
  api_version = "2024-02-01",
  azure_endpoint = "https://-m3f5c4y4-southcentralus.openai.azure.com/"
)


In [46]:

def summarize_clause(clause_text):
    messages = [
        {"role": "system", "content": "You are a helpful assistant specialized in summarizing legal clauses."},
        {"role": "user", "content": f"Please summarize the following clause: {clause_text}"}
    ]

    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )
    summary = response.choices[0].message.content.strip()
    return summary

In [47]:
def generate_summary(clause_title, relevant_clauses):
    prompt = f"Summarize the information about '{clause_title}' using the following references: {relevant_clauses}. If the information doesn't cover the clause title, indicate that it is absent."
    messages=[
            {"role": "system", "content": "You are a contract summarization assistant."},
            {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )
    summary = response.choices[0].message.content.strip()
    return summary


In [48]:
# List of clause titles you are looking for
'''
clause_titles = [
    "Termination Provisions",
    "Confidentiality",
    "Indemnification",
    "Rebates/Volume Discounts",
    "Payment Terms",
    "SLAs/Service Credits",
    "Price Protection",
    "Forex",
    "COLA/Inflation Adjustment",
    "Late Payment Penalties"
]
'''
clause_titles = [
    "Price Protection",
    "Termination Provisions",
    "Late Payment Penalties",
    "Invalid"
]

def generate_report(clause_titles):
    report = ""
    for title in clause_titles:
        clauses = search_for_clause(title)
        if clauses:
            report += f"Clause: {title}\n"
            summary = generate_summary(title, clauses)
            report += f"Summary: {summary}\n\n"
            '''
            for clause in clauses:
                summary = summarize_clause(clause)
                report += f"Summary: {summary}\n\n"
            '''
        else:
            report += f"Clause: {title} - Absent\n\n"
    return report

# Generate and print the final report
final_report = generate_report(clause_titles)
print(final_report)


Clause: Price Protection
Summary: The clause mentions 'Price Protection' and states that the prices mentioned in the agreement will remain fixed for 12 months. After that, the pricing will be reviewed annually based on market indices for comparable goods and services. The price adjustment will be limited to a maximum increase of 3% per year, unless mutually agreed upon. The Buyer has the right to request a formal pricing review in case of significant market fluctuations to ensure continued competitive rates. However, there is no information mentioned in the clause about the Supplier proposing anything.

Clause: Termination Provisions
Summary: Termination Provisions: Either party may terminate the agreement with 30 days' written notice. A penalty of $10,000 will be imposed if notice is not provided. 

Payment Schedule: Payments are to be made quarterly within 15 days of invoice receipt. Interest of 1.5% will be charged on late payments. 

Intellectual Property: All IP developed under th

In the same azure OpenAI resource with gpt4/35 deployment create test-embedding-ada-002 deployment

In [49]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [50]:
def generate_embeddings(text):
    response = client.embeddings.create(
        input=text, model="text-embedding-ada-002"
    )
    embeddings = [embedding.embedding for embedding in response.data]  # Correct way to access the embedding

    return embeddings

In [51]:
def retrieve_relevant_clause(question, clause_segments):
    # Generate embeddings for the question
    question_embedding = generate_embeddings([question])[0]

    # Generate embeddings for all clauses
    clause_embeddings = generate_embeddings([clause['Text'] for clause in clause_segments])

    # Compute cosine similarity between question embedding and clause embeddings
    similarities = cosine_similarity([question_embedding], clause_embeddings)

    # Find the most relevant clause based on the highest similarity score
    most_relevant_idx = np.argmax(similarities)

    return clause_segments[most_relevant_idx]


In [52]:

# Step 3: Use the retrieved clause to answer the question
def answer_question(question, relevant_clause):
    prompt = f"Given the following contract clause, answer the question: '{question}'\n\nClause: {relevant_clause['Text']}\n\nAnswer:"
    messages=[
            {"role": "system", "content": "You are a legal assistant."},
            {"role": "user", "content": prompt}
        ]
    # Call Azure OpenAI API to generate an answer
    response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
        max_tokens=150
    )

    return response.choices[0].message.content.strip()


In [62]:

# Example question about the contract
question = "confidentiality penalties?"

# Retrieve relevant clause
relevant_clause = retrieve_relevant_clause(question, clause_segments)

# Get the answer for the question
answer = answer_question(question, relevant_clause)

print(f"Answer: {answer}")

Answer: The clause provides an agreement between both parties not to disclose confidential information without written consent. If either party breaches this confidentiality agreement, there may be penalties in the form of up to $50,000. The specific period for this agreement is between January 1, 2024, and January 1, 2025.
